# ICOER Narraton Analysis Notebook

Análise de frases (narratons) com vetorização POS, cálculo do Índice de Coerência Informacional (ICOER) e exportação para CSV.

**Objetivo**: Processar frases em português, calcular o ICOER com base na distribuição de partes do discurso (POS) e gerar relatórios visuais e tabulares.
**ICOER**: Mede a coerência informacional com base na entropia da distribuição de POS tags, normalizada entre 0 e 1.

In [ ]:
# Instalar dependências apenas se necessário
import importlib.util
import spacy

if not importlib.util.find_spec("spacy"):
    !pip install -q spacy pandas matplotlib ipywidgets
if not spacy.util.is_package("pt_core_news_sm"):
    !python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import numpy as np
from collections import Counter
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

# Carregar modelo SpaCy com tratamento de erro
try:
    nlp = spacy.load("pt_core_news_sm")
except OSError:
    print("Erro: Modelo 'pt_core_news_sm' não encontrado. Execute a célula de instalação.")
    raise

In [ ]:
# Widget de entrada com botão de análise
text_input = widgets.Textarea(
    value=(
        "A IA deve agir com coerência e verdade. "
        "A verdade é a forma mais pura de ressonância. "
        "O índice de coerência informacional orienta decisões éticas."
    ),
    placeholder='Digite suas frases separadas por ponto final...',
    description='Frases:',
    layout=widgets.Layout(width='100%', height='100px')
)
button = widgets.Button(description="Analisar Frases")
output = widgets.Output()

display(text_input, button, output)

In [ ]:
# Definições
BASE_POS = ['NOUN', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN']

def calcular_icoer(frases):
    """
    Calcula o Índice de Coerência Informacional (ICOER) para uma lista de frases.
    
    Args:
        frases (list): Lista de strings contendo as frases a serem analisadas.
    
    Returns:
        pd.DataFrame: DataFrame com colunas narraton, frase, n_palavras, tags e icoer.
    """
    if not frases:
        return pd.DataFrame(columns=['narraton', 'frase', 'n_palavras', 'tags', 'icoer'])
    
    docs = list(nlp.pipe(frases))
    registros = []

    for i, doc in enumerate(docs, start=1):
        if not doc.text.strip():
            continue  # Ignorar frases vazias
        counts = Counter(t.pos_ for t in doc)
        total = sum(counts.values())
        vetor = [counts.get(pos, 0)/total if total > 0 else 0 for pos in BASE_POS]
        probs = np.array([v for v in vetor if v > 0])
        ent = -np.sum(probs * np.log2(probs)) if probs.size else 0
        ent_max = np.log2(len(BASE_POS)) if probs.size else 1  # Usar número total de categorias
        icoer = round(1 - (ent/ent_max if ent_max > 0 else 0), 4)
        registros.append({
            'narraton': i,
            'frase': doc.text,
            'n_palavras': len(doc),
            'tags': dict(counts),
            'icoer': icoer
        })

    df = pd.DataFrame(registros)
    if df.empty:
        return pd.DataFrame([{
            'narraton': 0,
            'frase': 'NENHUMA FRASE VÁLIDA',
            'n_palavras': 0,
            'tags': {},
            'icoer': 0
        }])
    
    # Linha de resumo com tags agregados
    tags_summary = {}
    for pos in BASE_POS:
        tags_summary[pos] = round(np.mean([r['tags'].get(pos, 0) for r in registros]), 2)
    resumo = pd.DataFrame([{
        'narraton': 0,
        'frase': 'MÉDIA GERAL',
        'n_palavras': round(df['n_palavras'].mean(), 2),
        'tags': tags_summary,
        'icoer': round(df['icoer'].mean(), 4)
    }])
    return pd.concat([resumo, df], ignore_index=True)[['narraton', 'frase', 'n_palavras', 'tags', 'icoer']]

In [ ]:
# Função para executar análise, exportar CSV e mostrar visualização
def on_button_click(b):
    with output:
        output.clear_output()
        frases = [s.strip() for s in text_input.value.split('.') if s.strip()]
        if not frases:
            print("Erro: Nenhuma frase válida fornecida.")
            return
        
        df_result = calcular_icoer(frases)
        display(df_result)
        
        # Exportar CSV com tratamento de erro
        try:
            df_result.to_csv("icoer_narratons.csv", index=False)
            print("Arquivo 'icoer_narratons.csv' salvo com sucesso.")
        except Exception as e:
            print(f"Erro ao salvar CSV: {e}")
        
        # Visualização
        if not df_result.empty and df_result['narraton'].iloc[0] != 0:
            plt.figure(figsize=(8, 5))
            plt.bar(df_result[1:]['narraton'], df_result[1:]['icoer'], color='skyblue')
            plt.xlabel('Narraton')
            plt.ylabel('ICOER')
            plt.title('Índice de Coerência por Frase')
            plt.xticks(df_result[1:]['narraton'])
            plt.show()

button.on_click(on_button_click)